# Before your start with this Tutorial

**Tutorial Intention:** Providing an example of iteration and related step on a modeling phase for you to:

*   Experience the data science lifecycle using Vectice
*   See how simple it is to connect your notebook to Vectice
*   Learn how to structure and log your work using Vectice

**Resources needed:**
*   <b>Tutorial Project: Forecast in-store unit sales (22.4)</b> - You can find it as part of your personal workspace named after your name
*   Dataset ready for modeling: https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/train_clean.csv
*   Vectice Webapp Documentation: https://docs.vectice.com/
*   Vectice API documentation: https://api-docs.vectice.com/sdk/index.html

## Installing Vectice

In [ ]:
!pip3 install --q vectice[git]
!pip3 show vectice

## Install optional packages for your project

In [ ]:
!pip3 install --q squarify
!pip3 install --q plotly

## Import libraries

In [ ]:
# importing mathematical and ds libraries
import pandas as pd  # data science essentials
import matplotlib.pyplot as plt  # essential graphical output
import numpy as np   # mathematical essentials
%matplotlib inline

# import Visual libraries
import plotly.offline as py
py.init_notebook_mode(connected=True)
from matplotlib import pyplot as plt
#import seaborn as sns  # enhanced graphical output

#importing other libraries
import IPython.display #this is for our data pipeline
import logging
logging.basicConfig(level=logging.INFO)

# Load scikit-learn packages for modeling
from sklearn.model_selection import train_test_split #Split function
from sklearn.linear_model import LinearRegression #LR function
from sklearn.metrics import mean_squared_error,mean_absolute_error

#import the Vectice Library
import vectice
from vectice import FileDataWrapper

##  Vectice Config     
- To log your work to Vectice, you need to connect your notebook to your profile using your personal API token       
- Click on your profile at the top right corner of the Vectice application --> API Tokens --> Create API Token       
- Provide a name and description for the key. We recommend you name the API Token: "Tutorial_API_Token" to avoid having to make additional changes to the notebook.
- Save it in a location accessible by this code

### Update the workspace name below to match the workspace name your project is in

In [ ]:
my_vectice = vectice.connect(config=r"Tutorial_API_token.json")
# print(my_vectice.workspaces) # print out a list of workspaces we have access to

my_workspace = my_vectice.workspace("Retail Ops") # replace workspace name
# print(my_workspace.projects) # print out a list of projects we have access to

my_project = my_workspace.project(".Forecast in-store unit sales")

## Capture milestones for the Modeling phase

In [ ]:
# We are looking to create a new model in our "Modeling" phase
modeling = my_project.phase("Modeling")

# Let's start working by starting a new iteration for the Modeling Phase
modeling_iter = modeling.iteration



In [ ]:
# Let's start working by starting a new iteration for the Modeling Phase
modeling_iter = modeling.iteration

#Let's have a look at all the steps defined in this phase
modeling_iter.step_names

In [ ]:
# Let's pick the first step
modeling_step = modeling_iter.step("Select Modeling Techniques")

# Here we are documenting the modeling technique that we will use on this iteration
modeling_step.close(message="For this first iteration we are going to use a Linear Regression model to get a base model.")

# Let's get the next step in the process
modeling_step = modeling_iter.step("Generate Test Design")

# Linear Regression Model

## Generate Test Design

* [Dataset ready for modeling](https://vectice-examples.s3.us-west-1.amazonaws.com/Tutorial/ForecastTutorial/train_clean.csv)          

     Alternatively, you can generate the dataset from the Data_Preparation code.

In [ ]:
#read the dataset
model_ds =pd.read_csv("train_clean.csv")

# Set split sizes
test_size = 0.20
# We will set the random seed so we always generate the same split.
random_state = 42

train, test = train_test_split(model_ds, test_size = test_size, random_state = random_state)

# Generate X_train, X_test, y_train, y_test, which we will need for modeling
X = model_ds.drop(['unit_sales'], axis=1)
y = model_ds["unit_sales"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

# Generate our 2 new datasets for training and testing
# train.to_csv("train_lr.csv")
# test.to_csv("test_lr.csv")

In [ ]:
model_ds=model_ds.fillna(0) 

In [ ]:
# Capture all the datasets that are used or generated by the model
my_project.origin_dataset = FileDataWrapper(path="train_clean.csv", name="Dataset ready for modeling")
my_project.origin_dataset = FileDataWrapper(path="train_lr.csv", name="Train Dataset for Linear Regression")
my_project.origin_dataset = FileDataWrapper(path="test_lr.csv", name="Test Dataset for Linear Regression")

In [ ]:
# Now that we completed this step, let's close it with a message to let our coworker know what we achieved in the webb app
# modeling_step.close(message="With our improved linear regression we expect better results on the RSME and the MAE as some tuning has been done and this is not a row linear regression anymore")
modeling_step = modeling_step.next_step(message="With our improved linear regression we expect better results on the RSME and the MAE as some tuning has been done and this is not a row linear regression anymore")

## Build Model

In [ ]:
# Get the Build Model step
# The next step was already acquired above no need to 
# modeling_step = modeling_iter.step("Build Model")

### Linear Regression

In [ ]:
#Lets create a linear regression model
model = LinearRegression()

model.fit(X_train.values, y_train.values)
    
pred = model.predict(X_test.values)

print(f"predicted responses:\n {pred}")
    
RMSE = np.sqrt(mean_squared_error(y_test.values, pred))
MAE = mean_absolute_error(y_test.values, pred)

print("root_mean_squared_error: ",RMSE) 
print("mean_absolute_error: ", MAE)

metrics = {"RMSE": RMSE, "MAE": MAE}

In [ ]:
plt.scatter(X_train.iloc[:,0].values, y_train ,color='g') 
plt.plot(X_test, pred,color='k') 
plt.savefig("regression_graph.png")
plt.show()

In [ ]:
# Let's log the model we trained along with its metrics, as a new version of the "Regression" model in Vectice.

# Get the Build Model step
# modeling_step = modeling_iter.step("Build Model")
modeling_iter.model = vectice.Model(name="Unit Sales Predictor", library="scikit-learn", technique="linear regression", metrics=metrics, attachments="regression_graph.png", predictor=model)
modeling_step.close(message="RMSE= " + str(metrics["RMSE"]) +  " and MAE= " + str(metrics["MAE"])) 

## Assess Model

In [ ]:
# Get the Collect Initial Data step
modeling_step = modeling_iter.step("Assess model")

In [ ]:
modeling_step.close(message="As expected the model performs better however this is not good enough and we should try a different method. We recommend doing a Random Forest as a new iteration")